In [15]:
"""
Heinrich.py
=======
Author - Daniel Monyak
12-6-24
=======

Source code for processing the Lund cohort data

1. Process clinical file
2. Import beta values
3. Filter samples
4. Calculate c_beta values for samples

"""

import pandas as pd
import numpy as np
import os
import PanCancerClock.src.util as nbl_util
nbl_consts = nbl_util.consts

proj_dir = os.path.join(nbl_consts['official_indir'], 'Heinrich')

## Create outdir if necessary
output_dir = os.path.join('outputs', 'Heinrich')
os.makedirs(output_dir, exist_ok=True)

## Beta values
beta_values = pd.read_table(os.path.join(proj_dir, 'beta_values.txt'), index_col=0)

print('Loaded.')

# print(f'{clinical.shape[0]} tumors initially')

# Remove second tumor from patient with two tumors
# Chose the tumor that would have been excluded anyway
# 1 was a second primary with mixed histology, 2 were metastases, 1 was lobular histology
# clinical = clinical.drop(['GSM1941866', 'GSM1941946', 'GSM1942009', 'GSM1941877'], axis=0)
# print(f'Manualy dropped 4 tumors from patient with multiple tumors')

# Only include primary tumors
# clinical['in_CpG_dataset'] = clinical['primary'] == 'primary'
# print(f'{clinical["in_CpG_dataset"].sum()} primary tumors')
# print(f'{clinical.loc[clinical["in_CpG_dataset"], "age"].unique().shape[0]} unique patients')

## Filter samples based on LUMP purity value
# Set in_CpG_dataset and reason_purity flags accordingly - based on LUMP value
# Remove impure tumors from beta_values

# LUMP threshold for this dataset
LUMP_THRESH = nbl_consts['LUMP_threshold']

LUMP_purity = nbl_util.getLUMP_values(beta_values)
# clinical = clinical.merge(LUMP_purity.rename('LUMP'), left_index=True, right_index=True)
# clinical['reason_purity'] = clinical['in_CpG_dataset'] & (clinical['LUMP'] < LUMP_THRESH)
# clinical['in_CpG_dataset'] &= ~clinical['reason_purity']
# print(f'{clinical["reason_purity"].sum()} tumors removed for having LUMP < {LUMP_THRESH}')


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [20]:
Clock_CpGs = np.loadtxt(os.path.join(nbl_consts['repo_dir'], 'Neuroblastoma', 'outputs', f'NBL_Clock_CpGs.txt'), dtype=str)

# Not all of the Clock sites are in this beta_values dataframe - wasn't exactly the 450k
Clock_CpGs = np.intersect1d(Clock_CpGs, beta_values.index)

# Don't need to remove any samples for having >= 5% missing values in Clock fCpGs
# assert (beta_values.loc[Clock_CpGs].isna().mean(axis=0) < 0.05).all()
# print('0 tumors had to be excluded for too many missing Clock beta values')

# beta_values = beta_values[clinical.index[clinical['in_CpG_dataset']]]
# print(f'{clinical["in_CpG_dataset"].sum()} tumors kept')

# Set in_analysis_dataset flag to indicate that tumor is ductal or not ductal
# clinical['in_analysis_dataset'] = clinical['in_CpG_dataset'] & (clinical['tumorType_loRes']=='ductal')
# n_removed_histology = clinical['in_CpG_dataset'].sum() - clinical['in_analysis_dataset'].sum()
# print(f'{n_removed_histology} tumors removed for having non-ductal histology')
# print(f'{clinical["in_analysis_dataset"].sum()} final tumors')

# Save clinical file
# clinical.to_csv(os.path.join(proj_dir, 'Lund.clinical.txt'), sep='\t')

print('Saved modified clinical file.')

## Calculate and save c_beta values for each tumor
c_beta = 1 - beta_values.loc[Clock_CpGs].std(axis=0)
c_beta.name = 'c_beta'
c_beta.to_csv(os.path.join(output_dir, 'Heinrich.NBL_sites.c_beta.txt'), sep='\t')

print('Saved c_beta value of each tumor.')


Saved modified clinical file.
Saved c_beta value of each tumor.
